# Compute i parameters

In [1]:
import brightway2 as bw
import pandas as pd
import os
import numpy as np

# Set working directry
path = "."
os.chdir(path)

# Local functions
from lookup_func import lookup_geothermal

# Set project
bw.projects.set_current("Geothermal")

# Retrieve activities
wellhead, diesel, steel, cement, water, \
drilling_mud, drill_wst, wells_closr, coll_pipe, \
plant, ORC_fluid, ORC_fluid_wst, diesel_stim, co2,_, _ = lookup_geothermal()
cooling_tower=bw.Database("geothermal energy").search("cooling tower")[0].key

list_act = [wellhead, diesel, steel, cement, water, 
         drilling_mud, drill_wst, wells_closr, coll_pipe,
         plant, cooling_tower, ORC_fluid, ORC_fluid_wst, diesel_stim]

# Retrieve methods 
ILCD_CC = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "climate change total" in str(method)]
ILCD_HH = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "human health" in str(method)]
ILCD_EQ = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "ecosystem quality" in str(method)]
ILCD_RE = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "resources" in str(method)]
ILCD = ILCD_CC + ILCD_HH + ILCD_EQ + ILCD_RE

# Calculate impact of activities
lca = bw.LCA({list_act[0]: 1}, ILCD[0])
lca.lci()
lca.lcia()
coeff = {}
for method in ILCD:
    s=[]
    lca.switch_method(method)
    for act in list_act:
        lca.redo_lcia({act: 1})
        s.append(lca.score)
    coeff[method] = s

# Retrieve CF for co2 emissions    
for method in ILCD:
    CFs = bw.Method(method).load()
    coeff[method].append(next((cf[1] for cf in CFs if cf[0]==co2),0))

# Build matrix
col_names = ["wellhead", "diesel", "steel", "cement", "water", \
         "drilling_mud", "drill_wst", "wells_closr", "coll_pipe", \
         "plant", "cooling tower", "ORC_fluid", "ORC_fluid_wst", "diesel_stim", "co2"]
coeff_matrix = pd.DataFrame.from_dict(coeff, orient="index", columns=col_names)

#Constant
drilling_waste_per_metre = 450

# Re-arrange matrix
coeff_matrix["concrete"] = coeff_matrix["cement"] + coeff_matrix["water"] * 1/0.65
coeff_matrix["ORC_fluid_tot"] = coeff_matrix["ORC_fluid"] - coeff_matrix["ORC_fluid_wst"]
coeff_matrix["electricity_stim"] = coeff_matrix["diesel_stim"] * 3.6 / 0.3
coeff_matrix["drill_wst"] = coeff_matrix["drill_wst"] * -1 * drilling_waste_per_metre
coeff_matrix=coeff_matrix.drop(columns=["cement", "ORC_fluid", "ORC_fluid_wst", "diesel_stim"])

col_ord = ["wellhead", "diesel", "steel", "concrete", \
         "drilling_mud", "drill_wst", "wells_closr", "coll_pipe", \
         "plant", "cooling tower", "ORC_fluid_tot", "water", "electricity_stim", "co2"]
coeff_matrix=coeff_matrix[col_ord]

is_= ["i1", "i2.1", "i2.2", "i2.3", "i2.4", "i2.5", \
      "i2.6","i3", "i4.1", "i4.2", "i4.3", "i5.1", "i5.2", "i6"]

if len(coeff_matrix.columns) == len(is_):
    coeff_matrix.columns=is_

In [2]:
coeff_matrix


,i1,i2.1,i2.2,i2.3,i2.4,i2.5,i2.6,i3,i4.1,i4.2,i4.3,i5.1,i5.2,i6
"(ILCD 2.0 2018 midpoint no LT, climate change, climate change total)",37611.449809,8.890842e-02,1.713816e+00,9.328241e-01,4.076658e+01,3.270302e+00,1.680112e+00,556.511500,1.298110e+05,2.830589e+05,1.953994e+01,7.051866e-04,3.146082e+00,1.0
"(ILCD 2.0 2018 midpoint no LT, human health, carcinogenic effects)",0.008744,1.417468e-10,5.724193e-07,2.301671e-09,4.966189e-07,4.229330e-05,7.453472e-09,0.000118,1.609336e-02,1.050642e-02,1.955774e-07,4.314110e-11,1.298980e-08,0.0
"(ILCD 2.0 2018 midpoint no LT, human health, ionising radiation)",938.386160,5.544320e-03,4.122670e-02,1.073440e-02,9.048418e-01,3.111033e-01,3.349863e-02,13.229952,4.044152e+03,2.136156e+04,2.645402e-01,2.900607e-05,1.922039e-01,0.0
"(ILCD 2.0 2018 midpoint no LT, human health, non-carcinogenic effects)",0.022281,1.931704e-09,1.316654e-06,4.500777e-08,5.103335e-06,1.311482e-04,1.005873e-07,0.000283,4.890049e-02,1.223006e-01,1.543690e-06,1.911262e-10,1.002894e-07,0.0
"(ILCD 2.0 2018 midpoint no LT, human health, ozone layer depletion)",0.002483,2.030752e-08,9.965641e-08,3.201636e-08,3.286511e-06,1.085525e-06,9.027492e-08,0.000035,1.241881e-02,1.471943e-01,7.844754e-04,3.786627e-11,6.997677e-07,0.0
"(ILCD 2.0 2018 midpoint no LT, human health, photochemical ozone creation)",177.381833,1.586389e-03,8.160569e-03,1.795630e-03,1.867750e-01,2.682373e-02,4.398159e-03,2.510056,5.476001e+02,8.594063e+02,4.310696e-02,2.217080e-06,5.459560e-02,0.0
"(ILCD 2.0 2018 midpoint no LT, human health, respiratory effects, inorganics)",0.003413,1.649275e-09,1.581448e-07,1.863021e-08,2.010977e-06,3.584068e-07,5.062763e-08,0.000046,9.350447e-03,1.377231e-02,6.157686e-07,3.850425e-11,6.373390e-08,0.0
"(ILCD 2.0 2018 midpoint no LT, ecosystem quality, freshwater and terrestrial acidification)",214.132990,1.254077e-03,9.432961e-03,2.375932e-03,1.890265e-01,2.589877e-02,5.531370e-03,3.281206,8.837621e+02,1.858586e+03,6.206367e-02,3.819721e-06,4.356839e-02,0.0
"(ILCD 2.0 2018 midpoint no LT, ecosystem quality, freshwater ecotoxicity)",108367.054092,1.148739e-02,6.743044e+00,1.205883e-01,2.832601e+01,2.357427e+03,4.794782e-01,1455.367162,2.534393e+05,3.030210e+05,8.577195e+00,7.011672e-04,5.488226e-01,0.0
"(ILCD 2.0 2018 midpoint no LT, ecosystem quality, freshwater eutrophication)",6.805882,3.345382e-07,4.351851e-04,1.214131e-05,2.606437e-03,6.862024e-03,3.093737e-05,0.094737,2.679558e+01,4.908280e+01,5.634018e-04,4.057804e-08,3.134290e-05,0.0


# Define symbolic

In [3]:
from sympy import symbols, collect, ratsimp, fraction

In [4]:
# Coefficients
i1, i2_1, i2_2, i2_3, i2_4, i2_5, i2_6, i3, i4, i4_1, i4_2, i4_3, i5_1, i5_2, i6 = \
symbols('i1, i2_1, i2_2, i2_3, i2_4, i2_5, i2_6, i3, i4, i4_1, i4_2, i4_3, i5_1, i5_2, i6')

# With or without success rate coefficients
W_nT, W_nTe = symbols('W_nT, W_nTe')

# Other variables that are actually irrelevant, because they will be multiplied by 0 in the enhanced model
# I wanna keep them though for the sake of completeness
E_co2 = symbols('E_co2')

# Variables needed in the enhanced models
W_d, D, P_ne = symbols('W_d, D, P_ne')

# Enhanced geothermal

In [5]:
# Klausen
from ege_klausen import parameters
parameters.static()   

SR_pi  = parameters["success_rate_primary_wells"] / 100
LT     = parameters["lifetime"]
SR_e   = parameters["success_rate_exploration_wells"] / 100
C_S    = parameters["specific_steel_consumption"]
C_C    = parameters["specific_cement_consumption"]
DM     = parameters["specific_drilling_mud_consumption"]
CP     = parameters["collection_pipelines"]
CF     = parameters["capacity_factor"]
A_p    = parameters["auxiliary_power"]
W_pi_n = parameters["number_of_wells"] 
SW_n   = np.round(0.5 + parameters["number_of_wells_stimulated_0to1"] * parameters["number_of_wells"])
S_w    = parameters["water_stimulation"]
S_el   = parameters["specific_electricity_stimulation"] / 1000
# Constants
CT_el  = 864
CT_n   = 7/303.3
W_e_n  = 3
W_e_en = W_e_n * 0.3
OF     = 300
  
class1 = ["carcinogenic effects", "non-carcinogenic effects", "respiratory effects, inorganics", \
          "freshwater ecotoxicity", "freshwater eutrophication", "dissipated water", 
          "land use", "minerals and metals"]

class2 = ["climate change total", "ionising radiation", "ozone layer depletion", \
          "photochemical ozone creation", "freshwater and terrestrial acidification", \
          "marine eutrophication", "terrestrial eutrophication", "fossils"]

In [6]:
# Supporting equations for enhanced plant wells
W_nT__wout_sr = W_pi_n + W_e_n
W_nT__with_sr = W_pi_n/SR_pi + W_e_en/SR_e
W_nTe_wout_expl = W_pi_n/SR_pi

# Main equation


In [7]:
nominator = W_nT*i1 \
          + W_d*W_nT * (D*i2_1 + C_S*i2_2 + C_C*i2_3  +  DM*i2_4 + i2_5 + i2_6) + \
          + W_nTe*CP*i3 \
          + P_ne*i4 \
          + SW_n*S_w * (i5_1 + S_el*i5_2)

c1 = P_ne*CF*(1-A_p)*LT*8760000
d1 = CT_el*CT_n*1000*30
denominator = c1  -  d1

summand = E_co2*i6

eq = nominator/denominator + summand
eq = eq.subs(i4, i4_1 + i4_2*CT_n + i4_3*OF)

In [8]:
eq

E_co2*i6 + (P_ne*(i4_1 + 0.0230794592812397*i4_2 + 300*i4_3) + W_d*W_nT*(D*i2_1 + 112.5*i2_2 + 58.0*i2_3 + 0.65*i2_4 + i2_5 + i2_6) + W_nT*i1 + 125.0*W_nTe*i3 + 70000000.0*i5_1 + 5250000.0*i5_2)/(189216000.0*P_ne - 598219.584569733)

In [9]:
eq_simplified_enhanced_0 = \
eq.subs([(W_nTe, W_nTe_wout_expl), # substitute with total number of wells without exploration
         (W_nT, W_nT__with_sr)])   # substitute with total number of wells with success rate

In [10]:
# The only variables left are P_ne, W_d, D and E_co2 (this one will be multiplied by i6=0, so is irrelevant)
eq_simplified_enhanced_0

E_co2*i6 + (P_ne*(i4_1 + 0.0230794592812397*i4_2 + 300*i4_3) + 5.49885675348346*W_d*(D*i2_1 + 112.5*i2_2 + 58.0*i2_3 + 0.65*i2_4 + i2_5 + i2_6) + 5.49885675348346*i1 + 519.884570149558*i3 + 70000000.0*i5_1 + 5250000.0*i5_2)/(189216000.0*P_ne - 598219.584569733)

### Compute chi

In [12]:
D_val = parameters["specific_diesel_consumption"] # set D to actual value
eq_simplified_enhanced_1 = eq_simplified_enhanced_0.subs([(D, D_val)])

In [15]:
chi_dict = {}
for method in ILCD:
    is_ = dict(coeff_matrix.T[method])
    is_dict = {k.replace('.', '_'): v for k,v in is_.items()}
    is_dict['i6'] = 0
    if method[2] in class1:
        eq_simplified_enhanced_chi = eq_simplified_enhanced_1.subs(is_dict)
        frac = fraction(eq_simplified_enhanced_chi)
        nom = collect(frac[0], ['P_ne','W_d'], evaluate=False)
        den = collect(frac[1], ['P_ne',], evaluate=False)
        chi1 = nom[W_d]
        chi2 = nom[P_ne]
        chi3 = nom[1]
        chi4 = den[P_ne]
        chi5 = -den[1]
        chi_dict[str(method)] = {'chi1': chi1,
                            'chi2': chi2,
                            'chi3': chi3,
                            'chi4': chi4,
                            'chi5': chi5}

In [16]:
# Example of the formula
eq_simplified_enhanced_chi

(1.95182304455899*P_ne + 0.0148718737802024*W_d + 28.0432786140078)/(189216000.0*P_ne - 598219.584569733)

In [17]:
chi_df = pd.DataFrame.from_dict(chi_dict, orient='index')
chi_df = chi_df.astype(float)
chi_df

,chi1,chi2,chi3,chi4,chi5
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'carcinogenic effects')",0.000596,0.016395,1.805251e-01,189216000.0,598219.58457
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'non-carcinogenic effects')",0.001659,0.052186,8.093133e-01,189216000.0,598219.58457
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'respiratory effects, inorganics')",0.000190,0.009853,3.800397e-01,189216000.0,598219.58457
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater ecotoxicity')",17813.809069,263006.012662,4.282918e+06,189216000.0,598219.58457
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater eutrophication')",0.335943,28.097410,2.540678e+02,189216000.0,598219.58457
"('ILCD 2.0 2018 midpoint no LT', 'resources', 'dissipated water')",519.734173,66475.844180,5.870490e+05,189216000.0,598219.58457
"('ILCD 2.0 2018 midpoint no LT', 'resources', 'land use')",10795.646774,840210.055761,1.521934e+07,189216000.0,598219.58457
"('ILCD 2.0 2018 midpoint no LT', 'resources', 'minerals and metals')",0.014872,1.951823,2.804328e+01,189216000.0,598219.58457


### Compute delta

In [18]:
W_d_val = parameters["average_depth_of_wells"] # set W_d to actual value
eq_simplified_enhanced_2 = eq_simplified_enhanced_0.subs([(W_d, W_d_val)])

In [19]:
delta_dict = {}
for method in ILCD:
    is_ = dict(coeff_matrix.T[method])
    is_dict = {k.replace('.', '_'): v for k,v in is_.items()}
    is_dict['i6'] = 0
    if method[2] in class2:
        eq_simplified_enhanced_delta = eq_simplified_enhanced_2.subs(is_dict)
        frac = fraction(eq_simplified_enhanced_delta)
        nom = collect(frac[0], ['P_ne','D'], evaluate=False)
        den = collect(frac[1], ['P_ne',], evaluate=False)
        delta1 = nom[D]
        delta2 = nom[P_ne]
        delta3 = nom[1]
        delta4 = den[P_ne]
        delta5 = -den[1]
        delta_dict[str(method)] = {'delta1': delta1,
                              'delta2': delta2,
                              'delta3': delta3,
                              'delta4': delta4,
                              'delta5': delta5}

In [20]:
# Example of the formula
eq_simplified_enhanced_delta

(29100.6002645126*D + 1965507.51899481*P_ne + 325629860.748546)/(189216000.0*P_ne - 598219.584569733)

In [21]:
delta_df = pd.DataFrame.from_dict(delta_dict, orient='index')
delta_df = delta_df.astype(float)
delta_df

,delta1,delta2,delta3,delta4,delta5
"('ILCD 2.0 2018 midpoint no LT', 'climate change', 'climate change total')",2077.802238,1.422058e+05,2.356767e+07,189216000.0,598219.58457
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'ionising radiation')",129.571539,4.616527e+03,1.167879e+06,189216000.0,598219.58457
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'ozone layer depletion')",0.000475,2.511586e-01,4.091086e+00,189216000.0,598219.58457
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'photochemical ozone creation')",37.074141,5.803668e+02,3.165185e+05,189216000.0,598219.58457
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater and terrestrial acidification')",29.307968,9.452764e+02,2.635118e+05,189216000.0,598219.58457
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'marine eutrophication')",12.928706,1.766491e+02,1.069355e+05,189216000.0,598219.58457
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'terrestrial eutrophication')",141.698461,1.729324e+03,1.176091e+06,189216000.0,598219.58457
"('ILCD 2.0 2018 midpoint no LT', 'resources', 'fossils')",29100.600265,1.965508e+06,3.256299e+08,189216000.0,598219.58457


# Save to excel

In [22]:
with pd.ExcelWriter(os.path.join(path, 'Simplified models coefficients enhanced - symbolic.xlsx')) as writer:
    chi_df.to_excel(writer, sheet_name='chi')
    delta_df.to_excel(writer, sheet_name='delta')

# Compare with analytical solution

In [23]:
coeffs_=pd.read_excel(os.path.join(path, "Simplified models coefficients - analytical.xlsx"), sheet_name=["alpha", "beta", "chi", "delta"], index_col=0, dtype=object)
chi_an = coeffs_["chi"]
delta_an = coeffs_["delta"]

In [24]:
chi_df - chi_an

,chi1,chi2,chi3,chi4,chi5
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'carcinogenic effects')",0,3.46945e-18,-2.77556e-17,-2.98023e-08,0
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'non-carcinogenic effects')",0,0,0,-2.98023e-08,0
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'respiratory effects, inorganics')",0,0,0,-2.98023e-08,0
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater ecotoxicity')",-3.63798e-12,0,0,-2.98023e-08,0
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater eutrophication')",0,3.55271e-15,2.84217e-14,-2.98023e-08,0
"('ILCD 2.0 2018 midpoint no LT', 'resources', 'dissipated water')",0,0,0,-2.98023e-08,0
"('ILCD 2.0 2018 midpoint no LT', 'resources', 'land use')",0,0,-1.86265e-09,-2.98023e-08,0
"('ILCD 2.0 2018 midpoint no LT', 'resources', 'minerals and metals')",-3.46945e-18,2.22045e-16,0,-2.98023e-08,0


In [25]:
delta_df-delta_an

,delta1,delta2,delta3,delta4,delta5
"('ILCD 2.0 2018 midpoint no LT', 'climate change', 'climate change total')",0,2.91038e-11,-3.72529e-09,-2.98023e-08,0
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'ionising radiation')",-2.84217e-14,0,-2.32831e-10,-2.98023e-08,0
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'ozone layer depletion')",5.42101e-20,0,0,-2.98023e-08,0
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'photochemical ozone creation')",0,0,0,-2.98023e-08,0
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater and terrestrial acidification')",3.55271e-15,0,-5.82077e-11,-2.98023e-08,0
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'marine eutrophication')",-1.77636e-15,0,2.91038e-11,-2.98023e-08,0
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'terrestrial eutrophication')",-2.84217e-14,4.54747e-13,2.32831e-10,-2.98023e-08,0
"('ILCD 2.0 2018 midpoint no LT', 'resources', 'fossils')",0,2.32831e-10,0,-2.98023e-08,0
